In [1]:
import pandas as pd
import os
import re
import string
from collections import defaultdict

In [2]:
data = []

for file in os.listdir('../data/insta_non_mentalhealth/'):
    print(file)
    if file[-3:] == "csv":
        try:
#             print('True')
            path = "../data/insta_non_mentalhealth/"+str(file)
            df = pd.read_csv(path)
            data.append(df)
        except Exception as e:
            print(e)
df = pd.concat(data, axis=0, ignore_index=True)
df.head()

world.csv
eucouncil.csv
README.md
abcwnn.csv
nytimesopinionart.csv
yoga.csv
daringplanet.csv
Error tokenizing data. C error: Buffer overflow caught - possible malformed input file.

hidden_shots_.csv
talentmoves.csv
nature.research.csv
healthyfoodvideos.csv
commonwealth_sec.csv
healthy.foodyss.csv
wanderlustfest.csv
afpsport.csv
ageofempires.csv
ptiphotos.csv
ageofempiresgame.csv
mitpics.csv
viceindia.csv
sciencechannel.csv
voxdotcom.csv
life.csv
scroll_in.csv
educationaboutearth.csv
oxford_uni.csv
merge.py
nature_africa.csv
aamaadmiparty.csv
pbsnature.csv


,Unnamed: 0,ID,shortcode,text,hashtags,comments,likes
0,1,NaN,NaN,Ethiopian Prime Minister Abiy Ahmed has been a...,ethiopia eritrea worldpeace peace nobelpeacepr...,|Wow .. I wish Modi Indian prime minister PM ...,27279.0
1,2,NaN,NaN,For the first time in 40 years Iranian women w...,iran iranianwomen iransoccer soccer worldcup w...,|That is a progress then.. we are almost in 20...,45503.0
2,3,NaN,NaN,A critically endangered eastern bongo was just...,easternbongo bongo animal animals cute chester...,|We need help breaking the record for most pen...,9145.0
3,4,NaN,NaN,Six elephants fell to their deaths after tryin...,thailand thai elephant elephants nationalpark ...,|This is what we need in our community’s. Peop...,68705.0
4,5,NaN,NaN,Wednesday marks one year since Washington Post...,jamalkhashoggi justiceforjamalkhashoggi journa...,|Привтда НаташулЯ Я уже как третий месЯц дома ...,12438.0


# Data Processing

In [5]:
#https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',str(text))

In [6]:
# appending first 25 comments into one big comment
def appendComments(comment):
  lst = comment.split("|")
  final_comment = ""
  for idx,entry in enumerate(lst):
    if idx == 25:
      break
    entry.strip()
    final_comment +=  entry + " "
  return final_comment

In [7]:
def cleanComment(comment):
  deEmojified_comment = deEmojify(comment)
  return appendComments(deEmojified_comment)

In [8]:
df["comments"] = df["comments"].apply(lambda x:cleanComment(x))

In [9]:
#combining comments and actual post
df["text_with_comments"] = df["text"]+ " "+ df["comments"]

In [10]:
# removing punctuations,and converting upper to lower case letters

punctuations = string.punctuation
table = punctuations.maketrans(punctuations+string.ascii_uppercase,
                               " "*len(punctuations)+string.ascii_lowercase,)

def cleanPosts(sentence):
  sentence.strip()
  sen = sentence.translate(table)
  return sen

def cleanHashTags(sentence):
  sentence.strip()
  sen = sentence.translate(table)
#   return "$start " + sen + " end$"
  return sen

In [11]:
post_df = df["text_with_comments"].apply(lambda w:cleanPosts(str(w)))
hashtag_df = df["hashtags"].apply(lambda w: cleanHashTags(str(w)))

#### Likes

In [12]:
# here we only consider hashtags that are appearing atleast 10 times
def count_hashtag_likes(all_posts_hashtags,all_posts_likes):
  hashtag_likesCount = defaultdict(int)
  hashtagAppearanceCount = defaultdict(int)
  hashtag_first10_likesCount = defaultdict(int)
  for hashtags,count in zip(all_posts_hashtags,all_posts_likes):
    hashtags = str(hashtags).split()
    for hashtag in hashtags:
      hashtagAppearanceCount[hashtag] += 1
      if hashtagAppearanceCount[hashtag] >= 10:
        if hashtag not in hashtag_likesCount:
          hashtag_likesCount[hashtag] = hashtag_first10_likesCount[hashtag] + count
        hashtag_likesCount[hashtag] += count
        continue
      hashtag_first10_likesCount[hashtag] += count      
  return hashtag_likesCount

In [13]:
hashtag_likesCount = count_hashtag_likes(hashtag_df.values.tolist(),df["likes"].values.tolist())
print(len(hashtag_likesCount))

2036


In [14]:
#  list of hashtags appearing atleast 10 times
frequent_hashtags = hashtag_likesCount.keys()
print(len(frequent_hashtags))

2036


#### Split Data

In [15]:
from sklearn.model_selection import train_test_split
X, X_val, Y, Y_val = train_test_split(post_df,hashtag_df, test_size=0.10,shuffle = True)
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.10,shuffle = True)

In [16]:
Y_train.head()

13687       ipl mumbaiindians csk chennaisuperkings cskvmi
4174     hidshots wanderfolk lifeofadventure igmasters ...
1002                        europeans rugby lgbt lgbtiq eu
1458     refugees migrants refugeecrisis migrationeu ju...
5054     healthyeating delicious healthylifestyle healt...
Name: hashtags, dtype: object

In [17]:
X_ftrain = pd.concat([X_train, X_val])
Y_ftrain = pd.concat([Y_train, Y_val])
frames = [X_ftrain, Y_ftrain]
train_df = pd.concat(frames, axis=1, join='inner')
# print(result.loc[[8737]])
print(len(train_df))
frames = [X_test, Y_test]
test_df = pd.concat(frames, axis=1, join='inner')
print(len(test_df))
train_df.head()

14790
1463


,text_with_comments,hashtags
13687,mumbai indians clinched their fourth title wit...,ipl mumbaiindians csk chennaisuperkings cskvmi
4174,pc visualsofjulius throwing boiling water i...,hidshots wanderfolk lifeofadventure igmasters ...
1002,in his inclusive rugby team gerald from irelan...,europeans rugby lgbt lgbtiq eu
1458,looking ahead to tomorrow s justice and home a...,refugees migrants refugeecrisis migrationeu ju...
5054,😋breakfast sausage guacamole🥑 stacks 💪🏼😍 qui...,healthyeating delicious healthylifestyle healt...


In [27]:
train_df = train_df.rename(
    columns={"text_with_comments": "input_text", "hashtags": "target_text"}
)
train_df.head()
test_df = test_df.rename(
    columns={"index":"prefix", "text_with_comments": "input_text", "hashtags": "target_text"}
)
test_df.head()

,input_text,target_text
13221,the eiffel tower officially opened 129 years a...,eiffeltower paris france
15864,no bjp4india no incindia no aamaadmiparty\n...,ripbravehearts pulwamaattack
2102,try day friday and they brought reena and i ...,nothanks wnn
2658,donald trump’s muslim ban is cowardly and dan...,art illustration muslimban
2315,to live and die in paris people have been dy...,art illustration death paris france funeral pl...


### Model Imports

In [19]:
import os
from datetime import datetime
import logging

import pandas as pd
from sklearn.model_selection import train_test_split
from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs

In [18]:
from random import randint
import pandas as pd
import requests, zipfile
try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO
import os

def download_data():
    if 'train_FD004.txt' not in os.listdir('data'):
        print('Downloading Data...')
        # Download the data
        r = requests.get("https://ti.arc.nasa.gov/c/6/", stream=True)
        z = zipfile.ZipFile(StringIO.StringIO(r.content))
        z.extractall('data')
    else:
        print('Using previously downloaded data')
    
def load_data(data_path):   
    operational_settings = ['operational_setting_{}'.format(i + 1) for i in range (3)]
    sensor_columns = ['sensor_measurement_{}'.format(i + 1) for i in range(26)]
    cols = ['engine_no', 'time_in_cycles'] + operational_settings + sensor_columns
    data = pd.read_csv(data_path, sep=' ', header=None, names=cols)
    data = data.drop(cols[-5:], axis=1)
    data['index'] = data.index
    data.index = data['index']
    data['time'] = pd.date_range('1/1/2000', periods=data.shape[0], freq='600s')
    print('Loaded data with:\n{} Recordings\n{} Engines'.format(
        data.shape[0], len(data['engine_no'].unique())))
    print('21 Sensor Measurements\n3 Operational Settings')
    return data

def new_labels(data, labels):
    ct_ids = []
    ct_times = []
    ct_labels = []
    data = data.copy()
    data['RUL'] = labels
    gb = data.groupby(['engine_no'])
    for engine_no_df in gb:
        instances = engine_no_df[1].shape[0]
        r = randint(5, instances - 1)
        ct_ids.append(engine_no_df[1].iloc[r,:]['engine_no'])
        ct_times.append(engine_no_df[1].iloc[r,:]['time'])
        ct_labels.append(engine_no_df[1].iloc[r,:]['RUL'])
    ct = pd.DataFrame({'engine_no': ct_ids,
                       'cutoff_time': ct_times,
                       'RUL': ct_labels})
    ct = ct[['engine_no', 'cutoff_time', 'RUL']]
    ct.index = ct['engine_no']
    ct.index = ct.index.rename('index')
    return ct

def make_cutoff_times(data):
    gb = data.groupby(['engine_no'])
    labels = []


    for engine_no_df in gb:
        instances = engine_no_df[1].shape[0]
        label = [instances - i - 1 for i in range(instances)]
        labels += label
    
    return new_labels(data, labels)

def feature_importances(X, reg, feats=5):
    feature_imps = [(imp, X.columns[i]) 
                    for i, imp in enumerate(reg.feature_importances_)]
    feature_imps.sort()
    feature_imps.reverse()
    for i, f in enumerate(feature_imps[0:feats]):
        print('{}: {} [{:.3f}]'.format(i + 1, f[1], f[0]))
    print('-----\n')
    return [f[1] for f in feature_imps[:feats]]


## Model

In [20]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.ERROR)

In [21]:
model_args = Seq2SeqArgs()
model_args.do_sample = True
model_args.eval_batch_size = 64
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 2500
model_args.evaluate_during_training_verbose = True
model_args.fp16 = False
model_args.learning_rate = 5e-5
model_args.max_length = 128
model_args.max_seq_length = 128
model_args.num_beams = None
model_args.num_return_sequences = 3
model_args.num_train_epochs = 2
model_args.overwrite_output_dir = True
model_args.reprocess_input_data = True
model_args.save_eval_checkpoints = False
model_args.save_steps = -1
model_args.top_k = 50
model_args.top_p = 0.95
model_args.train_batch_size = 8
model_args.use_multiprocessing = False
# model_args.use_cuda = False
model_args.wandb_project = "Paraphrasing with BART"

In [22]:
model = Seq2SeqModel(
    encoder_decoder_type="bart",
    encoder_decoder_name="facebook/bart-large",
    use_cuda=False,
    args=model_args,
)

In [ ]:
model.train_model(train_df, eval_data=test_df)

INFO:simpletransformers.seq2seq.seq2seq_utils: Creating features from dataset file at cache_dir/


INFO:simpletransformers.seq2seq.seq2seq_model: Training started


wandb: Currently logged in as: mmvv (use `wandb login --relogin` to force relogin)


In [29]:
to_predict = [
    str(input_text)
    for input_text in zip(test_df["input_text"].tolist())
]

In [ ]:
truth = test_df["target_text"].tolist()

preds = model.predict(to_predict)


In [ ]:
if not os.path.isdir('./results'):
    os.makedir('./results')
    
with open(f"results/predictions_{datetime.now()}.txt", "w") as f:
    for i, text in enumerate(test_df["input_text"].tolist()):
        f.write(str(text) + "\n\n")

        f.write("Truth:\n")
        f.write(truth[i] + "\n\n")

        f.write("Prediction:\n")
        for pred in preds[i]:
            f.write(str(pred) + "\n")
        f.write(
            "________________________________________________________________________________\n"
        )